### Dataset description

The dataset is focused on hospital-related information spanning the years 1999-2008, and it is available at https://archive.ics.uci.edu/ml/datasets/Diabetes+130-US+hospitals+for+years+1999-2008. It comprises over 50 features that represent both patient and hospital outcomes. The aim is to predict which patients will need hospital readmission.

In the dataset, there are three distinct classes used for readmission categorization:

1. `<30` indicates patients who were readmitted in less than 30 days.
2. `>30` represents patients who were readmitted in more than 30 days.
3. `No` signifies no record of readmission.

To streamline the classification task, we categorize patients into two groups: those with no record of readmission versus those with a record of readmission, achieved by consolidating the classes labeled as "<30" and ">30".

### Setup and Library Imports

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer


import warnings
warnings.filterwarnings("ignore")

### Loading Dataset and Initial Data Exploration

In [2]:
path = 'diabetic_data.csv'
dataset = pd.read_csv(path)

print(f'The shape of the dataset is: {dataset.shape}\n')
print(f'The type of the columns is:\n{dataset.dtypes.value_counts()}')

The shape of the dataset is: (101766, 50)

The type of the columns is:
object    37
int64     13
Name: count, dtype: int64
